In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
os.chdir('../')
os.getcwd()
import os,sys
sys.path.append('main/')
from shape_gradient import *
from cost_surface import *
import numpy as np
import logging
%load_ext line_profiler
import dask.array as da
np.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/users/home/robin/miniconda3/envs/shape_opti/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    

In [2]:
#from dask.distributed import Client,LocalCluster
#cluster = LocalCluster(n_workers=1)

In [3]:
#client=Client(cluster)

In [4]:
#import dask

In [5]:
#print(client.scheduler_info()['services'],client)
#client = Client(processes=False)

In [6]:
path_config_file='config_file/config_test.ini'
config = configparser.ConfigParser()
config.read(path_config_file)

['config_file/config_test.ini']

In [7]:
config['other']['dask']='True'
%lprun -u 1 -f cost_surface cost_surface(config)

Timer unit: 1 s

Total time: 3.37263 s
File: main/cost_surface.py
Function: cost_surface at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def cost_surface(config,S=None,Sp=None):
    10         1          0.0      0.0      0.0      if config['other']['dask']=='True':
    11         1          3.4      3.4    100.0          return cost_surface_dask(config,S,Sp)
    12                                               else:
    13                                                   return cost_surface_without_dask(config,S,Sp)

In [8]:
shape_grad=Shape_gradient(config=config)
S_parametrization=shape_grad.S_parametrization

In [9]:
%lprun -u 1 -f shape_grad.compute_gradient_df shape_grad.compute_gradient_df(S_parametrization)

Timer unit: 1 s

Total time: 68.4066 s
File: main/shape_gradient.py
Function: compute_gradient_df at line 160

Line #      Hits         Time  Per Hit   % Time  Line Contents
   160                                               def compute_gradient_df(self,paramS):
   161                                                   #compute the shape gradient by a differentiation first method
   162         1          0.0      0.0      0.0          result={}
   163         1          0.3      0.3      0.4          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
   164         1          2.1      2.1      3.1          theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
   165         1          0.4      0.4      0.6          T=tools.get_tensor_distance(S,self.Sp,self.rot_tensor)
   166                                                   #LS=tools.compute_LS(T,self.matrixd_phi,S.dpsi,self.rot_tensor,self.Sp.n)
   167         1          0.2      0.2      0.2          Qj=to

In [10]:
%lprun -u 1 -f shape_grad.compute_gradient_of shape_grad.compute_gradient_of(S_parametrization)

Timer unit: 1 s

Total time: 5.9646 s
File: main/shape_gradient.py
Function: compute_gradient_of at line 59

Line #      Hits         Time  Per Hit   % Time  Line Contents
    59                                               def compute_gradient_of(self,paramS):
    60                                                   #compute the shape gradient by a optimization first method
    61         1          0.0      0.0      0.0          result={}
    62         1          0.3      0.3      4.2          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    63                                                   # for latter, when we will implement GPU support
    64         1          0.0      0.0      0.0          f,f_np,get = lambda x : x,lambda x : x,lambda x : x
    65                                                   #tensors computations
    66         1          0.2      0.2      3.7          T=tools.get_tensor_distance(S,self.Sp,self.rot_tensor)
    67         1     